In [5]:
%%writefile task2a.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE stackoverflow_;

DROP TABLE IF EXISTS post_tags;

CREATE TABLE post_tags as
SELECT 
    year,
    LOWER(tags_exploded) as tags_exploded,
    count(1) as tag_count
FROM posts
    LATERAL VIEW EXPLODE(tags) lateral_table as tags_exploded
WHERE post_type_id = 1
GROUP BY year, tags_exploded;

Overwriting task2a.hql


In [22]:
%%writefile task2b.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE stackoverflow_;

WITH post_tags as
(SELECT 
    year,
    LOWER(tags_exploded) as tags_exploded,
    count(1) as tag_count
FROM posts
    LATERAL VIEW EXPLODE(tags) lateral_table as tags_exploded
WHERE post_type_id = 1
GROUP BY year, tags_exploded)

SELECT
    t2016.tags_exploded,
    t2016.ranking,
    t2009.ranking,
    t2016.tag_count,
    t2009.tag_count
FROM (
    SELECT
        year,
        tags_exploded,
        rank() over(order by tag_count DESC) as ranking,
        tag_count
        from post_tags
        where year=2016
) t2016
left join(
    SELECT
        year,
        tags_exploded,
        rank() over(order by tag_count DESC) as ranking,
        tag_count
        from post_tags
        where year=2009
) t2009
ON
    t2016.tags_exploded = t2009.tags_exploded
limit 10;
    

Overwriting task2b.hql


In [23]:
!hive -f task2b.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.527 seconds
Query ID = jovyan_20180619223131_8cf58b27-4394-49a9-95af-bba0014f891f
Total jobs = 4
Launching Job 1 out of 4
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1529433183114_0026, Tracking URL = http://397fc019605f:8088/proxy/application_1529433183114_0026/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1529433183114_0026
Hadoop job information for